In [ ]:
from weather_app.helpers.HandlerFactory import HandlerFactory

paths = ["../.env.local", "../.env.public"]

handler_factory = HandlerFactory(paths)

mongo_handler = handler_factory.get_mongo_handler()
weather_handler = handler_factory.get_weather_handler()

In [ ]:
mongo_handler.verify_connection()

In [ ]:
import time

## get a json for the last n days from london (spoiler it's been raining the whole time)
n = 4
start = "03/30/2025"
print(start)
weather_handler.get_weather_n_days_into_future_by_date('London', None, None, start, n)

In [ ]:
## get a json for the last 2 days from london (spoiler: it's been raining the whole time)
# working cities: London, Prague, New York, Berlin, Birmingham, Chicago, Paris
start = "04/03/2025"
place = 'Bronx'
print(start)
weather_handler.get_weather_n_days_into_past_by_date(place, None, None, start, 2)

In [ ]:
start = "04/03/2025"
place = None
beijing_lat = 39.9042
beijing_lon = 116.4074

print(start)
weather_handler.get_weather_n_days_into_past_by_date(None, beijing_lat, beijing_lon, start, 2)

In [ ]:
weather_handler.get_weather_by_interval('London', None, None, '03/30/2025', '04/03/2025')

In [ ]:
cities = [
    {"name": "Beijing", "lat": 39.9042, "lon": 116.4074},
    {"name": "Tokyo", "lat": 35.6895, "lon": 139.6917},
    {"name": "Paris", "lat": 48.8566, "lon": 2.3522},
    {"name": "New York", "lat": 40.7128, "lon": -74.0060},
    {"name": "London", "lat": 51.5074, "lon": -0.1278},
    {"name": "Moscow", "lat": 55.7558, "lon": 37.6173},
    {"name": "Sydney", "lat": -33.8688, "lon": 151.2093},
    {"name": "Mumbai", "lat": 19.0760, "lon": 72.8777},
    {"name": "Rio de Janeiro", "lat": -22.9068, "lon": -43.1729},
    {"name": "Cairo", "lat": 30.0444, "lon": 31.2357},
    {"name": "Los Angeles", "lat": 34.0522, "lon": -118.2437},
    {"name": "Chicago", "lat": 41.8781, "lon": -87.6298},
    {"name": "Toronto", "lat": 43.6532, "lon": -79.3832},
    {"name": "Mexico City", "lat": 19.4326, "lon": -99.1332},
    {"name": "Berlin", "lat": 52.5200, "lon": 13.4050},
    {"name": "Madrid", "lat": 40.4168, "lon": -3.7038},
    {"name": "Rome", "lat": 41.9028, "lon": 12.4964},
    {"name": "Istanbul", "lat": 41.0082, "lon": 28.9784},
    {"name": "Seoul", "lat": 37.5665, "lon": 126.9780},
    {"name": "Singapore", "lat": 1.3521, "lon": 103.8198},
    {"name": "Hong Kong", "lat": 22.3193, "lon": 114.1694},
    {"name": "Dubai", "lat": 25.2048, "lon": 55.2708},
    {"name": "Johannesburg", "lat": -26.2041, "lon": 28.0473},
    {"name": "Buenos Aires", "lat": -34.6037, "lon": -58.3816},
    {"name": "Lima", "lat": -12.0464, "lon": -77.0428},
    {"name": "Santiago", "lat": -33.4489, "lon": -70.6693},
    {"name": "Bangkok", "lat": 13.7563, "lon": 100.5018},
    {"name": "Kuala Lumpur", "lat": 3.1390, "lon": 101.6869},
    {"name": "Jakarta", "lat": -6.2088, "lon": 106.8456},
    {"name": "Manila", "lat": 14.5995, "lon": 120.9842},
    {"name": "Delhi", "lat": 28.7041, "lon": 77.1025},
    {"name": "Bogota", "lat": 4.7110, "lon": -74.0721},
    {"name": "Nairobi", "lat": -1.2921, "lon": 36.8219},
    {"name": "Lagos", "lat": 6.5244, "lon": 3.3792},
    {"name": "Athens", "lat": 37.9838, "lon": 23.7275},
    {"name": "Vienna", "lat": 48.2082, "lon": 16.3738},
    {"name": "Stockholm", "lat": 59.3293, "lon": 18.0686},
    {"name": "Zurich", "lat": 47.3769, "lon": 8.5417},
    {"name": "Amsterdam", "lat": 52.3676, "lon": 4.9041},
    {"name": "Brussels", "lat": 50.8503, "lon": 4.3517},
    {"name": "Lisbon", "lat": 38.7223, "lon": -9.1393},
    {"name": "Prague", "lat": 50.0755, "lon": 14.4378},
    {"name": "Budapest", "lat": 47.4979, "lon": 19.0402},
    {"name": "Warsaw", "lat": 52.2297, "lon": 21.0122},
    {"name": "Dublin", "lat": 53.3498, "lon": -6.2603},
    {"name": "Edinburgh", "lat": 55.9533, "lon": -3.1883},
    {"name": "Vancouver", "lat": 49.2827, "lon": -123.1207},
    {"name": "Auckland", "lat": -36.8485, "lon": 174.7633},
    {"name": "Montreal", "lat": 45.5017, "lon": -73.5673},
    {"name": "Cape Town", "lat": -33.9249, "lon": 18.4241}
]

In [ ]:
# open weather api scraper function
def update_weather_data_for_cities(weather_handler, mongo_handler, cities, start_date, end_date):
    """
    For each city in the list, retrieve weather data for the interval specified by start_date and end_date,
    then insert the resulting JSON into the MongoDB 'weather_data' collection.

    :param weather_handler: An instance of WeatherApiHandler.
    :param mongo_handler: An instance of MongoHandler.
    :param cities: List of dicts with keys "name", "lat", and "lon".
    :param start_date: Start date as a string in "mm/dd/yyyy" format.
    :param end_date: End date as a string in "mm/dd/yyyy" format.
    """
    for city in cities:
        lat = city["lat"]
        lon = city["lon"]
        try:
            # Call the API using the interval method; city is not used (pass empty string)
            weather_json = weather_handler.get_weather_by_interval("", lat, lon, start_date, end_date)
            # Optionally, you might want to add a field for the city name to the JSON:
            weather_json["city_name"] = city["name"]
            # Insert the JSON into the MongoDB collection "weather_data"
            mongo_handler.insert_document("weather_data", weather_json)
            print(f"Updated weather data for {city['name']}.")
        except Exception as e:
            print(f"Error updating data for {city['name']}: {e}")

In [ ]:
update_weather_data_for_cities(weather_handler, mongo_handler, cities, "03/26/2025", "04/03/2025") # pls dont run this again pls

In [ ]:
# get the last 5 days of data for beijing
beijing_lat = 39.9042
beijing_lon = 116.4074
start = "04/03/2025"

# we are in db_xd in collection weather_data
mongo_handler.get_document_count('weather_data')